In [232]:
import pandas as pd
import numpy as np
import re

In [233]:
df_2022 = pd.read_csv('../export/2022-test-results.csv', index_col= 0)
df_2023 = pd.read_csv('../export/2023-projections.csv', index_col=0)
df_adp = pd.read_csv('../export/2023-ave-draft-pick.csv', index_col=0)

dataset = data = pd.read_csv('../data/clean_2010-2011-2012-2013-2014-2015-2016-2017-2018-2019-2020-2021-2022-playerstats.csv', index_col=0)


## 2022 Test Data Analysis

#### Add Percent Error Column and Rank Difference

In [234]:
df_2022['PPR_PercentError'] = abs(df_2022["PPR_Projected_2022"] - df_2022["PPR_Actual_2022"]) / df_2022["PPR_Actual_2022"] * 100
df_2022["Rank_Difference"] = abs(df_2022["Projected_Rank"] - df_2022["Actual_Rank"])

#### Find players with greatest diff between projected and actual

Find players with high rank difference, and visualize in terms of actual rank

In [235]:
def find_percentile(percentile, array) :
    sorted_array = array.sort_values()
    
    percentile_x = np.percentile(sorted_array, percentile)
    
    return percentile_x

In [236]:
quantile90_rankdiff = find_percentile(90,df_2022['Rank_Difference'])

df_2022[df_2022['Rank_Difference'] >= quantile90_rankdiff].sort_values('Actual_Rank').head()

,Player,PPR_Projected_2022,PPR_Actual_2022,Projected_Rank,Actual_Rank,PPR_PercentError,Rank_Difference
56,Breece Hall,116.881486,115.1,143.0,150.5,1.547773,7.5
375,Kenyan Drake,102.012591,104.1,173.0,166.0,2.005196,7.0
13,Alexander Mattison,86.472092,88.4,201.0,192.0,2.180891,9.0
106,Chuba Hubbard,89.675278,87.7,190.0,197.0,2.252312,7.0
441,Melvin Gordon,89.473912,87.1,191.0,198.0,2.725502,7.0


Find 90th Percentile PPR Players with High PPR Percent Error

In [237]:
quantile90_actualppr = find_percentile(90, df_2022['PPR_Actual_2022'])
df_2022[df_2022['PPR_Actual_2022']>=quantile90_actualppr].sort_values('PPR_PercentError', ascending = False).head()

,Player,PPR_Projected_2022,PPR_Actual_2022,Projected_Rank,Actual_Rank,PPR_PercentError,Rank_Difference
348,Justin Jefferson,383.271583,368.7,3.0,5.0,3.952152,2.0
346,Justin Herbert,291.306082,281.3,21.0,24.0,3.557086,3.0
104,Christian McCaffrey,344.394469,356.4,7.0,6.0,3.368555,1.0
492,Patrick Mahomes,405.188438,417.4,1.0,1.0,2.925626,0.0
593,Travis Kelce,307.282217,316.3,13.0,12.0,2.851022,1.0


## 2023 Project Data Analysis

Clean data to remove roman numerals to prepare for merge

In [238]:
df_2023['Player'] = df_2023['Player'].apply(lambda x: re.sub(r'\s+(I|V|X|L|C|D|M)+$', '', x))
df_adp['Player'] = df_adp['Player'].apply(lambda x: re.sub(r'\s+(I|V|X|L|C|D|M)+$', '', x))


Add Team and Position Information

In [239]:
df_2023 = pd.merge(df_2023, 
                   dataset.drop_duplicates('Player', keep = 'last')[['Player','Tm', 'FantPos']], 
         on = 'Player',
         how = 'inner').rename({
    'Tm' : 'Team',
    'FantPos' : 'Position'
}, axis = 1)


Merge dataframe

In [240]:
df_2023 = pd.merge(df_2023, df_adp[['Player', 'AVG']], on = 'Player', how = 'right').sort_values('PPR_Projected_2023', ascending=False)

#### Calculate VORP

**Set Draft Parameters**

In [241]:
num_starters = {
    'QB': 1,
    'RB': 2,
    'WR': 2,
    'TE': 1,
}
num_teams = 12

Calculate Player VORP

In [242]:
baseline = {}

for pos, num in num_starters.items():
    baseline[pos] = df_2023[df_2023['Position'] == pos]['PPR_Projected_2023'].nlargest(num * num_teams).min()

df_2023['VORP'] = df_2023.apply(lambda row: row['PPR_Projected_2023'] - baseline.get(row['Position'], 0), axis=1)

#### Generate Draft Order

In [243]:
df_2023['Rank'] = df_2023['VORP'].rank(method = 'min', ascending = False)

Add columns for RankvADP

In [244]:
df_2023['RankvsADP'] = df_2023['AVG'] - df_2023['Rank']

Sort according to draft order

In [245]:
df_2023 = df_2023.sort_values('Rank').set_index('Rank')

### Write Result

Reorder columns

In [246]:
df_2023 = df_2023[df_2023.columns[[0,2,3,1,5,4,6]]]
df_2023.rename(columns = {
    'AVG' : 'ADP'
}, inplace = True)

In [247]:
df_2023.to_csv('../2023-draft-order.csv')